In [7]:
import os
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

In [8]:
def euler_to_quaternion(euler_angles):
    """
    Convert Euler angles (rotation3D) to quaternion [qx, qy, qz, qw].
    :param euler_angles: List or tuple of Euler angles [x, y, z] in radians.
    :return: Quaternion [qx, qy, qz, qw]
    """
    rotation = R.from_euler('xyz', euler_angles)
    quaternion = rotation.as_quat()  # Returns [qx, qy, qz, qw]
    return quaternion

In [9]:
def convert_to_target_format(source_json, primary_token):
    target_data = []
    
    for instance in source_json[0]["instances"]:
        center = instance["contour"]["center3D"]
        size = instance["contour"]["size3D"]
        rotation = instance["contour"]["rotation3D"]

        # Convert Euler angles to quaternion
        rotation_quaternion = euler_to_quaternion([rotation["x"], rotation["y"], rotation["z"]])

        target_data.append({
            'sample_token': primary_token,
            'translation': [
                center["x"],
                center["y"],
                center["z"]
            ],
            'size': [
                size["x"],
                size["y"],
                size["z"]
            ],
            'rotation': rotation_quaternion,
            'velocity': np.array([0, 0]),  # Default value
            'detection_name': instance["className"],
            'detection_score': 1.0,  # Default value
            'attribute_name': ''
        })
    
    return target_data

In [10]:
# Helper function to convert numpy arrays to lists
def convert_numpy_to_list(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, list):
        return [convert_numpy_to_list(item) for item in data]
    elif isinstance(data, dict):
        return {key: convert_numpy_to_list(value) for key, value in data.items()}
    return data

In [11]:
SOURCE_DIR = '/root/metrics/jsons/GT'
TARGET_DIR = '/root/metrics/converted-jsons/GT_converted'

In [12]:
os.makedirs(TARGET_DIR, exist_ok=True)

for file in os.listdir(SOURCE_DIR):
    # Read the source JSON file
    with open(os.path.join(SOURCE_DIR, file), 'r') as f:
        source_json = json.load(f)

    # Use file name as primary token considering there is a . in the file name
    primary_token = file.split('.')[0] + '.' + file.split('.')[1]

    # Convert the data to the target format
    converted_data = convert_to_target_format(source_json, primary_token)

    # Convert any numpy arrays to lists so they can be JSON serializable
    converted_data = convert_numpy_to_list(converted_data)

    target_file = os.path.join(TARGET_DIR, (str(primary_token) + '.json'))
    if not os.path.exists(target_file):
        target_data = {'results': {}}
    else:
        with open(target_file, 'r') as f:
            target_data = json.load(f)

    target_data['results'][primary_token] = converted_data

    # Write the target JSON file
    with open(target_file, 'w') as f:
        json.dump(target_data, f, indent=4)

    # Add the "meta" field at the same level as "results"
    target_data['meta'] = {
        "use_camera": False,
        "use_lidar": True,
        "use_radar": False,
        "use_map": False,
        "use_external": False
    }

    # Write the target JSON file
    with open(target_file, 'w') as f:
        json.dump(target_data, f, indent=4)